# Advanced Pandas

## Categorical Data

This introduces the 'Categorical' type in Pandas.

### Background and Motivation

Often, a column in a table may contain repeated instances of a smaller set of distinct values.

In [1]:
import numpy as np
import pandas as pd

In [2]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

Many data systems have special approaches for representing data with repeated values. In data warehousing, a method is to use dimension tables which contains the distinct values and storing the primary observations as integer keys referencing the observation;

In [6]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [7]:
dim

0     apple
1    orange
dtype: object

We can use the take method to restore the original Series of strings;

In [8]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

This type of representation is called categorical/dictionary-encoded representation.

The array of distinct values is called the categories/dictionary/levels of data.

The integer values that refernce the categories are called the category codes or simply just codes.

Categorical representation provides significant improvement in performance.

### Categorical type in pandas

Pandas has a special 'Categorical' type for integer-based categorical representation or encoding;

In [10]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits, 'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                   columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,3,1.939211
1,1,orange,11,2.096712
2,2,apple,8,0.532473
3,3,apple,7,1.219819
4,4,apple,12,0.422173
5,5,orange,3,1.017605
6,6,apple,3,1.643234
7,7,apple,6,1.810043


Here, df['fruit'] is an array of string objects. We can convert it to categorical tyoe by;

In [12]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

The values of fruit_cat are not a numpy array, but an instance of pandas.Categorical;

In [13]:
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

The 'Categorical' object has attributes 'categories' and 'codes';

In [14]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [15]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

We can convert a Dataframe column to categorical type as follows;

In [17]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

We can also create pandas.Categorical directly from other types of Python sequences;

In [18]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

If we have obtained codes from another source we can use from_codes constructor;

In [19]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

Categorical conversions assume no specific ordering of the categories. We can specify ordering in from_codes(ordered=True);

In [20]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

As we can see in the output '<' indicates the order. We can also convert an undordered one into an ordered one;

In [21]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

The categorical data need not be strings, they can be any immutable value type.

### Computaions with Categoricals

Using Categoricals in pandas is similar to using the normal non-encoded version.

Let us look at pandas.qcut binning function which returns pandas.Categorical;

In [22]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [24]:
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

Now adding some labels;|

In [25]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [27]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

The labeled bin categories do not have any info about the bin edges, so we can use groupyby to extract some info;

In [28]:
bins = pd.Series(bins, name='Quartiles')
bins

0      Q2
1      Q3
2      Q2
3      Q2
4      Q4
       ..
995    Q3
996    Q2
997    Q1
998    Q3
999    Q4
Name: Quartiles, Length: 1000, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [31]:
results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'min', 'max'])
          .reset_index())
results

,Quartiles,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


The 'quartile' column retains the original categoraical info;

In [32]:
results['Quartiles']

0    Q1
1    Q2
2    Q3
3    Q4
Name: Quartiles, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

We can get better performance gains using categoricals;

In [33]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))
categories = labels.astype('category')

In [34]:
labels.memory_usage()

80000128

In [35]:
categories.memory_usage()

10000332

As we can see the memenory usage is less.

The conversion to category is not very effecient, but it is a one-time cost; 

In [36]:
%time _ = labels.astype('category')

CPU times: total: 266 ms
Wall time: 536 ms


Groupyby operations can be significantly faster with categoricals.

### Categorical Methods

Categoricals have several special methods.

In [37]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

The special attribute 'cat' provides access to categorical methods;

In [38]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [39]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

We can use set_categories() to extend/alter the set of categories;

In [40]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

While the data appears to be unchanged, the new categories will be reflected in operations that use them;

In [41]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

In [42]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
dtype: int64

We can use remove_unused_categories() to trim unobserved categories;

In [43]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [44]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

We will often transform categorical data into dummy variables also known as one-hot encoding. This invloves creating a Dataframe with a column for each distinct category. These columns will have values 1 for occurences of a given category and 0 otherwise. We can use get_dummies() for this;

In [46]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## Advanced GroupBy use

### Group transforms and 'Unwarapped' GroupBys

We have seen the 'apply' method in grouped operations for performing transformations. There is another built-in method called transform whic h is similar but imposes more constraints on the kind of function we can use;
* It can produce a scalar value to be broadcast to the shape of the group
* It can produce an object of the same shape as the i/p group
* It must not mutate its i/p

In [47]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12)})
df

,key,value
0,a,0
1,b,1
2,c,2
3,a,3
4,b,4
5,c,5
6,a,6
7,b,7
8,c,8
9,a,9


Group means by key;

In [48]:
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

Suppose we wanted to produce a Series with the values replaced by the average grouped by 'key';

In [49]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

For built in finctions we can pass it as a string;

In [50]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

Transform also works with functions that return a Series, but the result must be the same size as input;

In [51]:
g.transform(lambda x: x * 2)

0      0
1      2
2      4
3      6
4      8
5     10
6     12
7     14
8     16
9     18
10    20
11    22
Name: value, dtype: int32

 We can compute the ranks in descending order for each group;

In [52]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

Consider a group transformation function composed from simple aggregations;

In [65]:
def normalize(x):
    return (x - x.mean()) / x.std()

We can obtain the same results for transform and apply in this case;

In [66]:
g.apply(lambda x: normalize(x))

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [67]:
g.transform(lambda x: normalize(x))

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

Built in aggregation functions like 'mean' or 'sum' are often much faster than apply function. These also have a 'fast past' when used with transform. This allows us to perform so-called unwrapped group operations;

In [68]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [69]:
normalized = ((df['value'] - g.transform('mean')) / g.transform('std'))
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## Techniques for method chaining

When doing transformations to a dataset we may create numerous temporary variables that are never used. Eg:

In [ ]:
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()

We can use DataFrame.assign as an alternative to column assigment;

In [ ]:
# Usual non-functional way
df2 = df.copy()
df2['k'] = v
# Functional assign way
df2 = df.assign(k=v)

'assign' allows for easier method chaining;

In [ ]:
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
         .groupby('key')
         .col1_demeaned.std())

While doing method chaining we may need to refer temporary objects, to help with this assign and other pandas functions accept function-like arguments called as 'callables';

In [ ]:
df = load_data()
df2 = df[df['col2'] < 0]
# This can be rewritten as:
df = (load_data()
     [lambda x: x['col2'] < 0])

Here, the result of load_data is not assigned to a variable, so the function passed into [] is then bound to the object at that stage of the method chain. We can write the entire sequence as a single cahined expression;

In [ ]:
result = (load_data()
         [lambda x: x.col2 < 0]
         .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
         .groupby('key')
         .col1_demeaned.std())

### The pipe method

When we need to use our own functions or 3rd-party library functions, the pipe method comes in. Eg:

In [ ]:
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)

When using functions that accept and return Series or DataFrame objects, you can rewrite this using calls to pipe:

In [ ]:
result = (df.pipe(f, arg1=v1)
         .pipe(g, v2, arg3=v3)
         .pipe(h, arg4=v4))

A potentially useful pattern for pipe is to generalize sequences of operations into reusable functions. Eg:

In [ ]:
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')

Suppose that you wanted to be able to demean more than one column and easily
change the group keys. Additionally, you might want to perform this transformation in a method chain;

In [ ]:
def group_demean(df, by, cols):
 result = df.copy()
 g = df.groupby(by)
 for c in cols:
 result[c] = df[c] - g[c].transform('mean')
 return result

Then it is possible to write;

In [ ]:
result = (df[df.col1 < 0]
         .pipe(group_demean, ['key1', 'key2'], ['col1']))